In [ ]:
-- Advanced query example: Compare efficiency between UNION ALL and JOIN approaches

-- This query demonstrates how to get similar results using both approaches
-- Sample query using the UNION ALL approach (current implementation)
SELECT
    pk,
    MIN(event_timestamp) AS first_event,
    MAX(event_timestamp) AS last_event,
    DATEDIFF('hour', MIN(event_timestamp), MAX(event_timestamp)) AS event_hours,
    COUNT(CASE WHEN source_table = 'slam_leg' THEN 1 END) AS slam_events,
    COUNT(CASE WHEN source_table = 'GMP' THEN 1 END) AS gmp_events,
    COUNT(CASE WHEN source_table = 'PSE' THEN 1 END) AS pse_events,
    MAX(was_delivered) AS was_delivered,
    MIN(CASE WHEN tertiary_status = 'DELIVERED' THEN event_timestamp END) AS delivery_time
FROM
    shipment_timeline
GROUP BY
    pk
HAVING
    COUNT(*) > 1
ORDER BY
    event_hours DESC
LIMIT 20;

-- NOTE: The equivalent JOIN approach would require more complex queries
-- to analyze event sequences and transitions, potentially using subqueries
-- for each source table and then joining them.

# Shipment Timeline Data Model

This notebook implements a consolidated shipment tracking data model that joins data from multiple sources.

In [ ]:
-- Step 1: Create a temp table for node mapping (locations)
DROP TABLE IF EXISTS amzl_mapping;
CREATE TEMP TABLE amzl_mapping AS (
    SELECT
        location_id AS node,
        timezone,
        region,
        country,
        GETDATE() AS run_time_utc,
        CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP) AS day_start_utc,
        CONVERT_TIMEZONE('UTC', timezone, GETDATE()) AS run_time_local,
        CONVERT_TIMEZONE('UTC', timezone, CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP)) AS local_offset_utc
    FROM
        "amzlanalytics"."perfectmile"."d_perfectmile_node_mapping_mdm"
    WHERE
        1=1
        AND location_status = 'A'
        AND country IN('US', 'CA')
        -- Uncomment to filter by specific regions if needed
        -- AND UPPER(region) IN('ROCKIES', 'UPSTATE NY') 
        AND (location_type = 'DS' 
            OR location_id IN ('MCO5', 'ZYG1', 'ZYN9', 'XVV2', 'XVV3', 'XYT6', 'XLC1', 'XVC1', 'XNK2')
        )
);

In [ ]:
-- Step 2: Create a temp table for slam legs

DROP TABLE IF EXISTS slam_leg;
CREATE TEMP TABLE slam_leg SORTKEY(shipment_key) DISTKEY (shipment_key) AS (
    SELECT a.*,
           'slam_leg' AS source_table,
           m.timezone,
           m.region,
           m.country,
           m.run_time_utc,
           m.day_start_utc,
           m.run_time_local,
           m.local_offset_utc
    FROM (
        SELECT
            shipment_id,
            package_id,
            shipment_id ||'#'|| package_id as shipment_key,
            route_id,
            route_warehouse_id,
            route_ship_method,
            route_internal_sort_code,
            route_external_sort_code,
            processing_date,
            leg_sequence_id,
            leg_id,
            leg_warehouse_id,
            leg_ship_method,
            leg_internal_sort_code,
            leg_external_sort_code,
            leg_destination_warehouse_id,
            ship_option,
            pickup_date,
            estimated_arrival_date,
            transit_time_in_hours,
            "zone",
            request_timestamp,
            dw_creation_date,
            GETDATE() AS unloaded_at,
            ROW_NUMBER() OVER (PARTITION BY shipment_id ORDER BY request_timestamp DESC, pickup_date DESC) AS leg_rn
        FROM backlog_datasets.ATROPS.o_slam_packages_leg_live
        WHERE request_timestamp > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
    ) a
    LEFT JOIN amzl_mapping m
    ON m.node = a.leg_warehouse_id
    WHERE a.leg_rn = 1
);

In [ ]:
DROP TABLE IF EXISTS gmp;
CREATE TEMP TABLE gmp INTERLEAVED SORTKEY (shipment_key, tracking_id) DISTKEY (tracking_id) AS (
SELECT
    shipment_type,
    sender_id,
    tracking_id,
    ship_track_event_code,
    standard_carrier_alpha_code,
    supplement_code,
    tcda_container_id,
    parent_container_id,
    parent_container_type,
    status_node_id,
    load_id,
    status_date,
    status_date_timezone,
    edi_standard_name,
    status_code,
    reason_code,
    fulfillment_reference_id,
    marketplace_id,
    amazon_bar_code,
    fulfillment_shipment_id,
    package_id,
    estimated_arrival_date,
    promised_arrival_date,
    tss_ship_date,
    business_unit,
    ship_option,
    ship_method,
    actual_delivery_date,
    attempted_delivery_date,
    is_export_charge_prepaid,
    is_virtual_scan,
    access_point_id,
    seller_id,
    transport_shipment_id,
    amazon_reference_number,
    additional_reference_number,
    status_ref_target_type,
    predicted_delivery_date,
    estimated_delivery_date,
    origin_address_country_code,
    manifest_id,
    service_type,
    dw_created_time,
    GETDATE() AS unloaded_at,
    fulfillment_shipment_id || '#' || package_id AS shipment_key,
    shipment_key || '#' || tracking_id as shipment_tracking_key,
    ROW_NUMBER() OVER(PARTITION BY tracking_id ORDER BY status_date DESC) as status_date_rn,
    1 as duplicate_rn,
    'GMP' as source_table
FROM
    "backlog_datasets"."amzlcore"."gmp_shipment_events_na" GMP
WHERE
    1=1
    AND GMP.dw_created_time > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
    AND (
        GMP.ship_track_event_code IN (
        'EVENT_108',    -- SWA Pickup Failure
        'EVENT_250',    -- Sender cancelled pick up (MFN?)
        'EVENT_253',    -- ATS Arrival
        'EVENT_254',    -- ATS Departure
        'EVENT_258',    -- Identify shipments that can be delivered beyond FRD and publish a new EDD to Customers
        'EVENT_301',    -- Delivery
        'EVENT_302',    -- Dispatch
        'EVENT_304',    -- Attempt
        'EVENT_407',    -- Recipient refused (MFN?)
        'EVENT_408',    -- Shipment is undeliverable and is returning to Amazon or to MFN seller
        'EVENT_476',    -- Potential lost
        'EVENT_760',    -- Shipment damaged and will be replaced
        'EVENT_762',    -- Missorted and will be returned
        'EVENT_764'     -- Mislabled and will be returned
        )
        OR
        (GMP.ship_track_event_code = 'EVENT_201' AND GMP.supplement_code = 'AMAZON_FACILITY')
    )
);

In [ ]:
DROP TABLE IF EXISTS pse;
CREATE TEMP TABLE pse INTERLEAVED SORTKEY (forward_tracking_id, state_status) DISTKEY (forward_tracking_id) AS (
SELECT
      PSE."type"
    , PSE.package_id
    , PSE.package_id_type
    , PSE.forward_amazon_barcode
    , PSE.forward_tracking_id
    , PSE.forward_tcda_container_id
    , PSE.state_location_type
    , PSE.state_location_id
    , PSE.state_location_destination_id
    , PSE.state_location_source_id
    , PSE.state_status
    , PSE.state_time
    , PSE.triggerer_id
    , PSE.triggerer_id_type
    , PSE.dw_created_time
    , PSE.state_sub_status
    , PSE.comp_type
    , PSE.comp_reason
    , PSE.comp_state
    , PSE.reverse_amazon_barcode
    , PSE.reverse_tcda_container_id
    , PSE.reverse_tracking_id
    , PSE.execution_id
    , PSE.execution_id_type
    , PSE.state_location_destination_type
    , PSE.state_location_source_type
    , ROW_NUMBER() OVER(PARTITION BY PSE.forward_tracking_id ORDER BY PSE.state_time DESC) AS state_time_rn
    , ROW_NUMBER() OVER(PARTITION BY forward_tcda_container_id, forward_tracking_id ORDER BY state_time DESC, reverse_tracking_id) AS forward_tcda_rn
    , 'PSE' as source_table
    , GETDATE() AS unloaded_at
    , 1 as duplicate_rn
FROM
    "backlog_datasets"."amzlcore"."package_systems_event_na" PSE
WHERE
    1=1
    AND dw_created_time > DATE_TRUNC('DAY', CONVERT_TIMEZONE('UTC', 'America/Chicago', GETDATE())) - INTERVAL '2 DAYS'
);

In [ ]:
DROP TABLE IF EXISTS node_type;
CREATE TABLE node_type SORTKEY (location_id) DISTKEY (location_id) AS (
SELECT
DISTINCT
      location_id
    , location_type
FROM
    "amzlanalytics"."perfectmile"."d_perfectmile_node_mapping_mdm"
WHERE 1=1
    AND location_id SIMILAR TO '[A-Z]{3}[A-Z0-9]'
ORDER BY
      location_type
    , location_id
);

## Shipment Timeline Data Model

The shipment timeline data model brings together data from three primary sources, stacking events to create a comprehensive timeline view of package movement:

1. **slam_leg** - Shipment legs from SLAM (Sort, Label, and Manifest) system
2. **gmp** - GMP (Global Marketplace Platform) shipment tracking events
3. **pse** - Package Systems Events 
4. **nc_load_summary_snapshot** - Vehicle routing data
5. **node_type** - Location type reference data

### Key Design Elements:

- **UNION ALL Structure**: Each event remains a separate row, stacked from different source systems
- **Data Filtering**:
  - Only includes GMP records with matching shipment_key in slam_leg
  - Only includes PSE records with matching tracking_id in GMP that matches slam_leg
  - This ensures we're only analyzing volume that's included in the slam_leg data
  
- **Common Key Fields**: 
  - `pk`: Common identifier (shipment_key or tracking_id) to track an item across systems
  - `source_table`: Identifies which system the record came from ('slam_leg', 'GMP', or 'PSE')
  - `event_timestamp`: Normalized timestamp from different source fields
  - `shipment_id`, `package_id`, `tracking_id`, `dw_created_time`: Normalized across sources

- **Location Type Mapping**:
  - Join with `node_type` for each location type (origin, status_node, destination)
  - Enhances data with location type information for richer analysis
  
- **Status Identification**:
  - Events can be classified in queries using CASE statements based on source_table, ship_track_event_code, and state_status
  - Allows for consistent status tracking across the entire shipment journey

### Optimization:

- **SORTKEY(pk, event_timestamp)**: Improves query performance for filtering and ordering by package and time
- **DISTKEY(pk)**: Distributes data efficiently across Redshift nodes by package identifier

### Common Use Cases:

1. Tracking a package's complete journey chronologically across multiple systems
2. Analyzing delivery performance metrics and time between key events
3. Identifying where packages may be getting delayed or lost
4. Creating comprehensive timelines of shipment events
5. Analyzing status transitions and time between key events
6. Correlating shipment events with vehicle routing data

### Data Modeling Approach: UNION ALL vs JOIN

This implementation uses a UNION ALL approach rather than a JOIN approach. Here's how they compare:

#### UNION ALL Approach (Current Implementation)
- **Structure**: Events remain as separate rows, stacked vertically
- **Advantages**:
  - Better for timeline analysis and event sequencing
  - Natural fit for tracking status transitions over time
  - Follows pattern used in waterfall_reference.sql
- **Considerations**:
  - More rows with sparse data (many NULL fields)
  - May use more storage due to NULLs for fields from other source tables

#### JOIN Approach (Alternative)
- **Structure**: Events from different sources combined into wide rows with many columns
- **Advantages**:
  - Better for retrieving all information about a package in a single row
  - May use less storage if there are many events per package
  - Potentially faster for queries that need all attributes at once
- **Considerations**:
  - More complex to analyze event sequences
  - JOIN operations can be expensive with large datasets
  - May duplicate events across joined rows

The UNION ALL approach was selected because it better supports the event timeline analysis needed for tracking package movement through the supply chain.

The example queries below demonstrate how to extract useful insights from this timeline view.

-- Step 3: Create a consolidated table that stacks events from slam_leg, gmp, and pse tables

DROP TABLE IF EXISTS consolidated;
CREATE TEMP TABLE consolidated SORTKEY(pk, event_timestamp) DISTKEY(pk) AS (
    -- SLAM_LEG events
    SELECT
        sl.shipment_key AS pk,
        'slam_leg' AS source_table,
        sl.request_timestamp AS event_timestamp,
        sl.shipment_id,
        sl.package_id,
        NULL AS tracking_id,
        sl.dw_creation_date AS dw_created_time,
        
        -- slam_leg specific fields
        sl.route_id,
        sl.route_warehouse_id,
        sl.route_ship_method,
        sl.route_internal_sort_code,
        sl.route_external_sort_code,
        sl.processing_date,
        sl.leg_sequence_id,
        sl.leg_id,
        sl.leg_warehouse_id,
        sl.leg_ship_method,
        sl.leg_internal_sort_code,
        sl.leg_external_sort_code,
        sl.leg_destination_warehouse_id,
        sl.ship_option,
        sl.pickup_date,
        sl.estimated_arrival_date,
        sl.transit_time_in_hours,
        sl.zone,
        sl.request_timestamp AS leg_request_timestamp,
        sl.timezone,
        sl.region,
        sl.country,
        
        -- Empty GMP fields
        NULL AS shipment_type,
        NULL AS sender_id,
        NULL AS ship_track_event_code,
        NULL AS standard_carrier_alpha_code,
        NULL AS supplement_code,
        NULL AS tcda_container_id,
        NULL AS parent_container_id,
        NULL AS parent_container_type,
        NULL AS status_node_id,
        NULL AS load_id,
        NULL AS status_date,
        NULL AS status_date_timezone,
        NULL AS edi_standard_name,
        NULL AS status_code,
        NULL AS reason_code,
        NULL AS fulfillment_reference_id,
        NULL AS marketplace_id,
        NULL AS amazon_bar_code,
        NULL AS promised_arrival_date,
        NULL AS tss_ship_date,
        NULL AS business_unit,
        NULL AS gmp_ship_option,
        NULL AS gmp_ship_method,
        NULL AS actual_delivery_date,
        NULL AS attempted_delivery_date,
        NULL AS is_export_charge_prepaid,
        NULL AS is_virtual_scan,
        NULL AS access_point_id,
        NULL AS seller_id,
        NULL AS transport_shipment_id,
        NULL AS amazon_reference_number,
        NULL AS additional_reference_number,
        NULL AS status_ref_target_type,
        NULL AS predicted_delivery_date,
        NULL AS estimated_delivery_date,
        NULL AS origin_address_country_code,
        NULL AS manifest_id,
        NULL AS service_type,
        
        -- Empty PSE fields
        NULL AS type,
        NULL AS pse_package_id,
        NULL AS package_id_type,
        NULL AS forward_amazon_barcode,
        NULL AS forward_tcda_container_id,
        NULL AS state_location_type,
        NULL AS state_location_id,
        NULL AS state_location_destination_id,
        NULL AS state_location_source_id,
        NULL AS state_status,
        NULL AS state_time,
        NULL AS triggerer_id,
        NULL AS triggerer_id_type,
        NULL AS state_sub_status,
        NULL AS comp_type,
        NULL AS comp_reason,
        NULL AS comp_state,
        NULL AS reverse_amazon_barcode,
        NULL AS reverse_tcda_container_id,
        NULL AS reverse_tracking_id,
        NULL AS execution_id,
        NULL AS execution_id_type,
        NULL AS state_location_destination_type,
        NULL AS state_location_source_type,
        
        -- nc_load_summary and node_type fields
        NULL AS origin,
        NULL AS final_destination,
        NULL AS origin_type,
        NULL AS destination_type,
        NULL AS origin_location_type,
        NULL AS status_node_location_type,
        NULL AS destination_location_type
    FROM 
        slam_leg sl
    
    UNION ALL
    
    -- GMP events - Only include records with matching shipment_key in slam_leg
    SELECT
        g.shipment_key AS pk,
        'GMP' AS source_table,
        g.status_date AS event_timestamp,
        g.fulfillment_shipment_id AS shipment_id,
        g.package_id,
        g.tracking_id,
        g.dw_created_time,
        
        -- Empty slam_leg fields
        NULL AS route_id,
        NULL AS route_warehouse_id,
        NULL AS route_ship_method,
        NULL AS route_internal_sort_code,
        NULL AS route_external_sort_code,
        NULL AS processing_date,
        NULL AS leg_sequence_id,
        NULL AS leg_id,
        NULL AS leg_warehouse_id,
        NULL AS leg_ship_method,
        NULL AS leg_internal_sort_code,
        NULL AS leg_external_sort_code,
        NULL AS leg_destination_warehouse_id,
        NULL AS ship_option,
        NULL AS pickup_date,
        NULL AS estimated_arrival_date,
        NULL AS transit_time_in_hours,
        NULL AS zone,
        NULL AS leg_request_timestamp,
        NULL AS timezone,
        NULL AS region,
        NULL AS country,
        
        -- GMP specific fields
        g.shipment_type,
        g.sender_id,
        g.ship_track_event_code,
        g.standard_carrier_alpha_code,
        g.supplement_code,
        g.tcda_container_id,
        g.parent_container_id,
        g.parent_container_type,
        g.status_node_id,
        g.load_id,
        g.status_date,
        g.status_date_timezone,
        g.edi_standard_name,
        g.status_code,
        g.reason_code,
        g.fulfillment_reference_id,
        g.marketplace_id,
        g.amazon_bar_code,
        g.promised_arrival_date,
        g.tss_ship_date,
        g.business_unit,
        g.ship_option AS gmp_ship_option,
        g.ship_method AS gmp_ship_method,
        g.actual_delivery_date,
        g.attempted_delivery_date,
        g.is_export_charge_prepaid,
        g.is_virtual_scan,
        g.access_point_id,
        g.seller_id,
        g.transport_shipment_id,
        g.amazon_reference_number,
        g.additional_reference_number,
        g.status_ref_target_type,
        g.predicted_delivery_date,
        g.estimated_delivery_date,
        g.origin_address_country_code,
        g.manifest_id,
        g.service_type,
        
        -- Empty PSE fields
        NULL AS type,
        NULL AS pse_package_id,
        NULL AS package_id_type,
        NULL AS forward_amazon_barcode,
        NULL AS forward_tcda_container_id,
        NULL AS state_location_type,
        NULL AS state_location_id,
        NULL AS state_location_destination_id,
        NULL AS state_location_source_id,
        NULL AS state_status,
        NULL AS state_time,
        NULL AS triggerer_id,
        NULL AS triggerer_id_type,
        NULL AS state_sub_status,
        NULL AS comp_type,
        NULL AS comp_reason,
        NULL AS comp_state,
        NULL AS reverse_amazon_barcode,
        NULL AS reverse_tcda_container_id,
        NULL AS reverse_tracking_id,
        NULL AS execution_id,
        NULL AS execution_id_type,
        NULL AS state_location_destination_type,
        NULL AS state_location_source_type,
        
        -- nc_load_summary and node_type fields from VLS join
        v.origin,
        v.final_destination,
        v.origin_type,
        v.destination_type,
        origin_nt.location_type AS origin_location_type,
        status_nt.location_type AS status_node_location_type,
        dest_nt.location_type AS destination_location_type
    FROM 
        gmp g
    -- Join with nc_load_summary_snapshot using parent_container_id (vrid)
    LEFT JOIN 
        nc_load_summary_snapshot v ON g.parent_container_id = v.vrid
    -- Join with node_type for location type mappings
    LEFT JOIN 
        node_type origin_nt ON v.origin = origin_nt.location_id
    LEFT JOIN 
        node_type status_nt ON g.status_node_id = status_nt.location_id
    LEFT JOIN 
        node_type dest_nt ON v.final_destination = dest_nt.location_id
    -- Only include GMP records with matching shipment_key in slam_leg
    WHERE
        EXISTS (
            SELECT 1
            FROM slam_leg sl
            WHERE sl.shipment_key = g.shipment_key
        )
    
    UNION ALL
    
    -- PSE events - Only include records with matching tracking_id in GMP that matches slam_leg
    SELECT
        COALESCE(p.forward_tracking_id, p.package_id) AS pk,
        'PSE' AS source_table,
        p.state_time AS event_timestamp,
        NULL AS shipment_id,
        NULL AS package_id,
        p.forward_tracking_id AS tracking_id,
        p.dw_created_time,
        
        -- Empty slam_leg fields
        NULL AS route_id,
        NULL AS route_warehouse_id,
        NULL AS route_ship_method,
        NULL AS route_internal_sort_code,
        NULL AS route_external_sort_code,
        NULL AS processing_date,
        NULL AS leg_sequence_id,
        NULL AS leg_id,
        NULL AS leg_warehouse_id,
        NULL AS leg_ship_method,
        NULL AS leg_internal_sort_code,
        NULL AS leg_external_sort_code,
        NULL AS leg_destination_warehouse_id,
        NULL AS ship_option,
        NULL AS pickup_date,
        NULL AS estimated_arrival_date,
        NULL AS transit_time_in_hours,
        NULL AS zone,
        NULL AS leg_request_timestamp,
        NULL AS timezone,
        NULL AS region,
        NULL AS country,
        
        -- Empty GMP fields
        NULL AS shipment_type,
        NULL AS sender_id,
        NULL AS ship_track_event_code,
        NULL AS standard_carrier_alpha_code,
        NULL AS supplement_code,
        NULL AS tcda_container_id,
        NULL AS parent_container_id,
        NULL AS parent_container_type,
        NULL AS status_node_id,
        NULL AS load_id,
        NULL AS status_date,
        NULL AS status_date_timezone,
        NULL AS edi_standard_name,
        NULL AS status_code,
        NULL AS reason_code,
        NULL AS fulfillment_reference_id,
        NULL AS marketplace_id,
        NULL AS amazon_bar_code,
        NULL AS promised_arrival_date,
        NULL AS tss_ship_date,
        NULL AS business_unit,
        NULL AS gmp_ship_option,
        NULL AS gmp_ship_method,
        NULL AS actual_delivery_date,
        NULL AS attempted_delivery_date,
        NULL AS is_export_charge_prepaid,
        NULL AS is_virtual_scan,
        NULL AS access_point_id,
        NULL AS seller_id,
        NULL AS transport_shipment_id,
        NULL AS amazon_reference_number,
        NULL AS additional_reference_number,
        NULL AS status_ref_target_type,
        NULL AS predicted_delivery_date,
        NULL AS estimated_delivery_date,
        NULL AS origin_address_country_code,
        NULL AS manifest_id,
        NULL AS service_type,
        
        -- PSE specific fields
        p.type,
        p.package_id AS pse_package_id,
        p.package_id_type,
        p.forward_amazon_barcode,
        p.forward_tcda_container_id,
        p.state_location_type,
        p.state_location_id,
        p.state_location_destination_id,
        p.state_location_source_id,
        p.state_status,
        p.state_time,
        p.triggerer_id,
        p.triggerer_id_type,
        p.state_sub_status,
        p.comp_type,
        p.comp_reason,
        p.comp_state,
        p.reverse_amazon_barcode,
        p.reverse_tcda_container_id,
        p.reverse_tracking_id,
        p.execution_id,
        p.execution_id_type,
        p.state_location_destination_type,
        p.state_location_source_type,
        
        -- nc_load_summary and node_type fields - empty for PSE
        NULL AS origin,
        NULL AS final_destination,
        NULL AS origin_type,
        NULL AS destination_type,
        NULL AS origin_location_type,
        NULL AS status_node_location_type,
        NULL AS destination_location_type
    FROM 
        pse p
    -- Only include PSE records with matching tracking_id in GMP that matches slam_leg
    WHERE
        EXISTS (
            SELECT 1
            FROM gmp g
            JOIN slam_leg sl ON sl.shipment_key = g.shipment_key
            WHERE g.tracking_id = p.forward_tracking_id
        )
);

In [ ]:
-- Step 4: Example query to show how to use the consolidated table with all events

-- Sample query 1: Get event timeline for a specific package
SELECT
    pk,
    source_table,
    event_timestamp,
    shipment_id,
    package_id,
    pse_package_id,
    tracking_id,
    status_node_id,
    state_location_id,
    state_status,
    ship_track_event_code
FROM
    consolidated
ORDER BY
    pk,
    event_timestamp
LIMIT 100;

-- Sample query 2: Get event counts by source
SELECT
    source_table,
    COUNT(*) AS event_count
FROM
    consolidated
GROUP BY
    source_table;

-- Sample query 3: Find packages with delivery events
SELECT
    c.pk,
    MAX(CASE WHEN c.ship_track_event_code = 'EVENT_301' OR c.state_status = 'DELIVERED' THEN 1 ELSE 0 END) AS has_delivered_event,
    COUNT(DISTINCT c.source_table) AS source_count
FROM
    consolidated c
GROUP BY
    c.pk
HAVING
    MAX(CASE WHEN c.ship_track_event_code = 'EVENT_301' OR c.state_status = 'DELIVERED' THEN 1 ELSE 0 END) = 1
LIMIT 10;

In [ ]:
-- Sample query 4: Analyze shipments by status progression
SELECT
    pk,
    MIN(CASE WHEN source_table = 'slam_leg' THEN event_timestamp ELSE NULL END) AS slam_time,
    MIN(CASE WHEN state_status = 'INDUCTED' THEN event_timestamp ELSE NULL END) AS induction_time,
    MIN(CASE WHEN state_status = 'STOWED' THEN event_timestamp ELSE NULL END) AS stow_time,
    MIN(CASE WHEN state_status = 'PICKED' THEN event_timestamp ELSE NULL END) AS pick_time,
    MIN(CASE WHEN state_status = 'STAGED' THEN event_timestamp ELSE NULL END) AS stage_time,
    MIN(CASE WHEN ship_track_event_code = 'EVENT_302' THEN event_timestamp ELSE NULL END) AS dispatch_time,
    MIN(CASE WHEN ship_track_event_code = 'EVENT_301' OR state_status = 'DELIVERED' THEN event_timestamp ELSE NULL END) AS delivery_time,
    COUNT(*) AS total_events
FROM
    consolidated
GROUP BY
    pk
HAVING
    COUNT(*) > 3  -- Only packages with at least 4 events
ORDER BY
    slam_time DESC
LIMIT 100;

-- Sample query 5: Find shipments with specific status transitions
WITH status_events AS (
    SELECT 
        pk,
        event_timestamp,
        CASE
            WHEN source_table = 'slam_leg' THEN 'SLAMMED'
            WHEN ship_track_event_code = 'EVENT_302' THEN 'OUT_FOR_DELIVERY'
            WHEN ship_track_event_code = 'EVENT_301' OR state_status = 'DELIVERED' THEN 'DELIVERED'
            ELSE 'OTHER'
        END AS event_status,
        ROW_NUMBER() OVER(PARTITION BY pk ORDER BY event_timestamp) AS event_num
    FROM 
        consolidated
    WHERE 
        source_table = 'slam_leg'
        OR ship_track_event_code IN ('EVENT_301', 'EVENT_302')
        OR state_status = 'DELIVERED'
)
SELECT
    s1.pk,
    s1.event_timestamp AS slam_time,
    s2.event_timestamp AS dispatch_time,
    s3.event_timestamp AS delivery_time,
    DATEDIFF('minute', s1.event_timestamp, s2.event_timestamp) AS minutes_to_dispatch,
    DATEDIFF('minute', s2.event_timestamp, s3.event_timestamp) AS minutes_to_deliver
FROM
    status_events s1
JOIN 
    status_events s2 ON s1.pk = s2.pk AND s1.event_num < s2.event_num AND s1.event_status = 'SLAMMED' AND s2.event_status = 'OUT_FOR_DELIVERY'
JOIN 
    status_events s3 ON s2.pk = s3.pk AND s2.event_num < s3.event_num AND s3.event_status = 'DELIVERED'
ORDER BY
    minutes_to_deliver DESC
LIMIT 50;

In [ ]:
-- Sample query 4: Analyze shipments by status progression
SELECT
    pk,
    MIN(CASE WHEN tertiary_status = 'SLAMMED' THEN event_timestamp ELSE NULL END) AS slam_time,
    MIN(CASE WHEN tertiary_status = 'INDUCTED' THEN event_timestamp ELSE NULL END) AS induction_time,
    MIN(CASE WHEN tertiary_status = 'STOWED' THEN event_timestamp ELSE NULL END) AS stow_time,
    MIN(CASE WHEN tertiary_status = 'PICKED' THEN event_timestamp ELSE NULL END) AS pick_time,
    MIN(CASE WHEN tertiary_status = 'STAGED' THEN event_timestamp ELSE NULL END) AS stage_time,
    MIN(CASE WHEN tertiary_status = 'OUT_FOR_DELIVERY' THEN event_timestamp ELSE NULL END) AS dispatch_time,
    MIN(CASE WHEN tertiary_status = 'DELIVERED' THEN event_timestamp ELSE NULL END) AS delivery_time,
    MAX(was_delivered) AS was_delivered,
    COUNT(*) AS total_events
FROM
    shipment_timeline
GROUP BY
    pk
HAVING
    COUNT(*) > 3  -- Only packages with at least 4 events
ORDER BY
    slam_time DESC
LIMIT 100;

-- Sample query 5: Find shipments with specific status transitions
WITH status_events AS (
    SELECT 
        pk,
        event_timestamp,
        tertiary_status,
        ROW_NUMBER() OVER(PARTITION BY pk ORDER BY event_timestamp) AS event_num
    FROM 
        shipment_timeline
    WHERE 
        tertiary_status IN ('SLAMMED', 'OUT_FOR_DELIVERY', 'DELIVERED')
)
SELECT
    s1.pk,
    s1.event_timestamp AS slam_time,
    s2.event_timestamp AS dispatch_time,
    s3.event_timestamp AS delivery_time,
    DATEDIFF('minute', s1.event_timestamp, s2.event_timestamp) AS minutes_to_dispatch,
    DATEDIFF('minute', s2.event_timestamp, s3.event_timestamp) AS minutes_to_deliver
FROM
    status_events s1
JOIN 
    status_events s2 ON s1.pk = s2.pk AND s1.event_num < s2.event_num AND s1.tertiary_status = 'SLAMMED' AND s2.tertiary_status = 'OUT_FOR_DELIVERY'
JOIN 
    status_events s3 ON s2.pk = s3.pk AND s2.event_num < s3.event_num AND s3.tertiary_status = 'DELIVERED'
ORDER BY
    minutes_to_deliver DESC
LIMIT 50;